In [6]:
%load_ext autoreload
%autoreload 2

from webbrowser import get
import pandas as pd
from sklearn.metrics import accuracy_score, r2_score
from models import *
import logging
from pretty_logger import get_logger
from pathlib import Path


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# define subset (ignore all other data)
ema = [f"Y{i}" for i in range(1, 8, 1)]
physical = [f"P{i}" for i in range(1, 5, 1)]
social = [f"S{i}" for i in range(1, 8, 1)]

In [8]:
df = pd.read_csv(datafile)
df["date"] = pd.to_datetime(df["day_survey"])
df["C"] = df["date"].apply(
    lambda date: (
        COVIDStatus.POST_COVID if date > date_covid else COVIDStatus.PRE_COVID
    )
)
df_head = df.head(5).copy()

df.rename(columns=reverse_ema_dictionary, inplace=True)
df.set_index(["uid", "date"], inplace=True)

subset = ema + physical + social + sleep + ["C"]
df = df[subset]
df.dropna(inplace=True)

sets_df = pd.read_parquet(sets_file, engine="pyarrow")

In [9]:
skip_emas = list(set(ema_dictionary.keys()).difference(set(["Y2", "Y3"])))
skip_treatments = list(
    set(subset).difference(
        set(
            [
                "P1",
                "P4",
                # "S1",
                # "S2",
                "S3",
                # "S4",
                "S5",
                "S6",
                # "Z1",
                # "Z2",
                # "Z3",
            ]
        )
    )
)

skip_treatments

['Y4',
 'Y1',
 'Z3',
 'Y6',
 'P2',
 'Z1',
 'S1',
 'Y2',
 'Y5',
 'S4',
 'S2',
 'C',
 'Z2',
 'S7',
 'Y7',
 'Y3',
 'P3']

In [10]:
# index = 17
# model_row = sets_df.iloc[index]

alphas = [0.2, 1, 2]
n_estimatorss = [400]

for n_estimators in n_estimatorss:
    for alpha in alphas:
        for index, model_row in sets_df.iterrows():
            already_fitted_sets = []

            covariate_set = CovariateSet(
                row=model_row,
                data=df,
                outcomes_to_skip=skip_emas,
                treatments_to_skip=skip_treatments,
            )

            if not covariate_set.valid_set:
                logger.debug(f"Skipping {covariate_set} (no valid set)")
                continue

            if covariate_set.set_to_fit in already_fitted_sets:
                logger.info(
                    f"Skipping {covariate_set} (already did {covariate_set.set_to_fit})"
                )
                continue
            already_fitted_sets.append(covariate_set.set_to_fit)

            logger.info(f"Fitting\n{covariate_set!r}")
            logger.info(
                f"Median of {covariate_set.treatment} {df[covariate_set.treatment].median()}"
            )
            wbm = WBKernelModel(
                data=df,
                treatment=covariate_set.treatment,
                outcome=covariate_set.outcome,
                separating_set=covariate_set.restricted_adjustment_set,
                name=f"row:{index}",
            )
            logger.info(
                f"pre_rsq train={wbm.pre_r_squared[0]}, pre_rsq test={wbm.pre_r_squared[1]}\n"
                f"pre mae test ={wbm.pre_r_squared[2]}\n"
                f"post_rsq train={wbm.post_r_squared[0]}, post_rsq test={wbm.post_r_squared[1]}\n"
            )
            logger.info(
                "-----------------------------------------------------\n"
            )

[2024-05-28 16:10:10.644] - modelslog- INFO N/A- 1895262791.py: 30 [<module>]: Fitting
treatment: P1:excercise (seconds), outcome: Y2:phq4_score, adjustment set={'P4', 'P2', 'S1', 'S4', 'S3', 'S6', 'S2'}
[2024-05-28 16:10:10.646] - modelslog- INFO N/A- 1895262791.py: 31 [<module>]: Median of P1 10.9634
[2024-05-28 16:11:31.186] - modelslog- INFO N/A- 1895262791.py: 41 [<module>]: pre_rsq train=0.5439242796475374, pre_rsq test=-0.8098204271218932
pre mae test =2.254140081441697
post_rsq train=0.5171966643519755, post_rsq test=-0.8515877005201655

[2024-05-28 16:11:31.187] - modelslog- INFO N/A- 1895262791.py: 46 [<module>]: -----------------------------------------------------

[2024-05-28 16:11:31.188] - modelslog- INFO N/A- 1895262791.py: 30 [<module>]: Fitting
treatment: P4:sports (hours), outcome: Y2:phq4_score, adjustment set={'P2', 'S1', 'S4', 'S5', 'S3', 'P3'}
[2024-05-28 16:11:31.190] - modelslog- INFO N/A- 1895262791.py: 31 [<module>]: Median of P4 0.08325
